In [1]:
!pip install websocket-client
!pip install rel

     ---------------------------------------- 57.3/57.3 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import json
import random
import requests
import websocket
import _thread
import time
import json
import rel
from ipyleaflet import Map, GeoJSON, TileLayer

basemapUrl = "https://cartocdn-gusc.global.ssl.fastly.net/opmbuilder/api/v1/map/named/opm-mars-basemap-v0-1/all/{z}/{x}/{y}.png" 
basemap = TileLayer(url=basemapUrl, zoom=3, tms=False)

hiriseUrl = "http://s3-eu-west-1.amazonaws.com/whereonmars.cartodb.net/celestia_mars-shaded-16k_global/{z}/{x}/{y}.png" 
hirise = TileLayer(url=hiriseUrl, zoom=3,  tms=True)

vikingUrl = "http://s3-eu-west-1.amazonaws.com/whereonmars.cartodb.net/viking_mdim21_global/{z}/{x}/{y}.png"
viking = TileLayer(url=vikingUrl, zoom=3, tms=True)

m = Map(zoom=3)
m.add_layer(basemap)
m.add_layer(hirise)
m.add_layer(viking)

def projectToGround(a):
   a["geometry"]["coordinates"] = [ [c[0], c[1]] for c in a["geometry"]["coordinates"]][0]
   return a


def normalizePointAnnotations(annotations):
   annotations["features"] = [ projectToGround(a) for a in annotations["features"] if a["geometry"]["type"] == "Point"]
   return annotations

def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

class Annotations:
  def __init__(self):
     self.__layer = None
  
  def set_layer(self,v):
     m.remove_layer(self.__layer)
     m.add_layer(v)
     self.__layer = v


annotations = Annotations()

def attachToPRo3D():
    def on_message(ws, message):
        print("received incremental annotation lookup")
        # for now, we fetch the whole list of annotations
        currentAnnotations = json.loads(requests.get('http://localhost:4321/api/integration/geojson_latlon').content)
        geo_json = GeoJSON(
            data = normalizePointAnnotations(currentAnnotations)
        )
        annotations.set_layer(geo_json)
        ws.send("blub")

    def on_error(ws, error):
        print(error)

    def on_close(ws, close_status_code, close_msg):
        print("### closed ###")

    def on_open(ws):
        print("Opened connection")



    ws = websocket.WebSocketApp("ws://localhost:4321/api/integration/ws/geojson_xyz",
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)

    ws.run_forever(dispatcher=rel, reconnect=1)
    rel.signal(2, rel.abort)
    rel.dispatch()  


def queryPRo3D():
   annotationsAsString = requests.get('http://localhost:4321/api/integration/geojson_latlon').content
   currentAnnotations = json.loads(annotationsAsString)

   m.center = (-2.04694183792199, -5.4913)
   geo_json = GeoJSON(
       data=normalizePointAnnotations(currentAnnotations),
       style={
           'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
       },
       hover_style={
           'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
       },
       style_callback=random_color
   )
   m.zoom = 15
   m.add_layer(geo_json)

m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [3]:
queryPRo3D()

In [3]:
attachToPRo3D()

Opened connection
received incremental annotation lookup
A)
None
C
B)
received incremental annotation lookup
A)
GeoJSON(data={'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-5.498144896617236, -2.0532835399628175]}}]})
C
B)
received incremental annotation lookup
A)
GeoJSON(data={'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-5.494347026829018, -2.0513729165532957]}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-5.498144896617236, -2.0532835399628175]}}]})
C
B)
received incremental annotation lookup
A)
GeoJSON(data={'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-5.494347026829018, -2.0513729165532957]}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-5.498144896617236, -2.0532835399628175]}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-5.500